go to https://account.polar.com and download your data

In [1]:
#TODO: parse the timezone offset from the excercizes and use that
#also change timestamp parsing to iso
import os
workingDataFile = "/home/chowder/Documents/workingData/polar/hr/polarHRdf.parquet.gzip"

# Get the list of all files and directories
exportPath = "/home/chowder/Documents/dataExports/polar/25-10-24/"
exportfn = "polar-user-data-export_00bf6058-7016-47b4-98ae-ee6629425140"
dir_list = os.listdir(exportPath + exportfn)

In [ ]:
hrFilenames = sorted([x for x in dir_list if x.split("-")[0] == "training"])
hrFilenames[0]

'training-session-2025-08-22-8175259254-51ee623d-b4f9-495e-9e0a-5aed1d7c2100.json'

In [3]:
import json
data = json.load(open(exportPath + exportfn + "/" + hrFilenames[0]))
data["exercises"][0]["samples"]['heartRate'][0]
# exercises

{'dateTime': '2025-08-22T12:15:03.000'}

In [ ]:
#last time it took 130mintes to run and had 25.6M rows
import json
import pandas as pd

columnNames = ["sampleDT", "value"]

samplesCount = 0
samplesList = []

for fn in hrFilenames:
    print(fn)
    data = json.load(open(exportPath + exportfn + "/" + fn))

    samples = data["exercises"][0]["samples"]
    if 'heartRate' in samples:
        for row in samples['heartRate']:
            if len(row) == 2:
                samplesList.append([pd.to_datetime(row["dateTime"] + "-0700"), row["value"]])
                samplesCount += 1
                if samplesCount % 20_000 == 0:
                    print(samplesCount)


training-session-2025-08-22-8175259254-51ee623d-b4f9-495e-9e0a-5aed1d7c2100.json
20000
training-session-2025-03-15-8074835785-daa211ee-1067-443a-89b4-ffc8ec8b6dfc.json
40000
60000
80000
100000
120000
140000
160000
180000
200000
training-session-2024-10-13-7980610900-b303eb3c-499f-4197-b0ce-89c18dfa46f6.json
220000
240000
260000
280000
training-session-2025-01-03-8027841788-fde19d35-eaa3-4de9-a0db-9a20d59e7cfe.json
300000
320000
340000
360000
380000
training-session-2024-09-13-7961134032-f494d8be-d3cb-47ec-89af-342ada986287.json
400000
420000
440000
460000
480000
500000
training-session-2025-02-18-8059544674-09afca84-a540-442e-9968-52d819599c75.json
520000
540000
560000
580000
training-session-2025-01-09-8033635108-cd721ea6-4f35-4984-af38-abb19cd9dde4.json
600000
620000
640000
660000
680000
700000
720000
740000
training-session-2025-06-28-8143069718-ffc7a599-1a42-47ef-9bc8-c8496cacb7af.json
760000
780000
training-session-2025-08-25-8177633291-1c01bd36-8280-420b-86aa-930034273aeb.json
80

In [5]:
samplesList = sorted(samplesList, key=lambda x:x[0])
samplesList = [x for x in samplesList if x[1] > 0]

In [6]:
import pandas as pd

polarHRdf = pd.DataFrame(data=samplesList, columns=columnNames)

polarHRdf = polarHRdf.set_index("sampleDT")

In [7]:
polarHRdf

,value
sampleDT,
2019-10-27 15:32:45-07:00,69
2019-10-27 15:32:46-07:00,69
2019-10-27 15:32:47-07:00,69
2019-10-27 15:32:48-07:00,69
2019-10-27 15:32:49-07:00,69
...,...
2025-09-08 14:38:56-07:00,70
2025-09-08 14:38:57-07:00,70
2025-09-08 14:38:58-07:00,70


In [8]:
polarHRdf.dtypes

value    int64
dtype: object

In [10]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")+1]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath)
from rwWorkingTSDf import writeWorkingTSDf

In [12]:
writeWorkingTSDf(["abhik", "0", "polar", "h10", "hr", "builtin"], polarHRdf)

hashes match for 2019-10-27T223245,000000+0000_2024-08-25T201034,000000+0000_d885ae27_.parquet.gzip
hashes match for 2024-08-25T201034,000000+0000_2024-09-04T075519,000000+0000_0e029fa7_.parquet.gzip
hashes match for 2024-09-04T075519,000000+0000_2024-09-11T143909,000000+0000_d4f9005e_.parquet.gzip
hashes match for 2024-09-11T143909,000000+0000_2024-09-17T125137,000000+0000_de80b603_.parquet.gzip
hashes match for 2024-09-17T125137,000000+0000_2024-09-24T114403,000000+0000_c2a8ac67_.parquet.gzip
hashes match for 2024-09-24T114403,000000+0000_2024-09-30T175934,000000+0000_d7fee0e9_.parquet.gzip
hashes match for 2024-09-30T175934,000000+0000_2024-10-06T132538,000000+0000_a3d21d88_.parquet.gzip
hashes match for 2024-10-06T132538,000000+0000_2024-10-12T213035,000000+0000_0eeeb8d4_.parquet.gzip
hashes match for 2024-10-12T213035,000000+0000_2024-10-21T190646,000000+0000_50f91d6c_.parquet.gzip
hashes match for 2024-10-21T190646,000000+0000_2024-10-30T085134,000000+0000_bdb4c002_.parquet.gzip


In [ ]:
#polarHRdf.to_parquet(workingDataFile,
#              compression='gzip') 